# Common Time Series Models: AR, MA, ARMA, ARIMA

In [ ]:
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf

%matplotlib inline

**Baseline observations**

We create some time series observations from processes that we already know the parameters to.

We will use these as a baseline throughout the lecture.

In [ ]:
T = 100

rho, sigma = 0.7, 0.1

# Simulate white noise
wn = sigma*np.random.randn(T)

# Simulate a random walk
rw = np.cumsum(sigma*np.random.randn(T))


@numba.jit(nopython=True)
def simulate_ar1(T, rho=0.9, sigma=0.1):
    out = np.zeros(T)
    
    for t in range(1, T):
        out[t] = rho*out[t-1] + sigma*np.random.randn()
    
    return out

# Simulate an AR(1)
ar = simulate_ar1(T, rho, sigma)

## Autocorrelation function and partial autocorrelation function

We'll start today by talking about the [autocorrelation function](https://en.wikipedia.org/wiki/Autocorrelation) and [partial autocorrelation function](https://en.wikipedia.org/wiki/Partial_autocorrelation_function).


We defined autocorrelation previously to be

$$\rho_{t, s} = \text{Corr}(Y_t, Y_s)$$

For a stationary process, $\rho_{t, s} = \rho_{0, s-t}$ so we will typically refer to the correlation as a "lagged" value so

$$\rho_\tau = \rho_{0, \tau} = \rho_{t, t + \tau}$$

and we'll define the $N$ lag autocorrelation function to be

$$\text{ACF}(N) = \begin{bmatrix}\rho_{0} & \rho_1 & \dots & \rho_N \end{bmatrix}$$

**Partial autocorrelation function**

The autocorrelation function simply measures the correlation between the random variables that are $\tau$ apart from one another.

Consider the following process:

$$y_{t} = \begin{cases} y_{t-1} \; \text{if $t > 0$}\\ \varepsilon_{t} \; \text{if $t = 0$} \end{cases}$$

The correlation between any of the lags would be 1, but, in some ways, this doesn't really tell us much... All of the "interesting" correlation happens at the first lag.

For two elements of the sequence of random variables, $Y_t$ and $Y_s$ ($t<s$), the partial autocorrelation measures the correlation between $Y_t$ and $\hat{Y}_s$ where $\hat{Y}_s$ is $Y_s$ with all of the linear explantory power of $\{Y_{t+1}, Y_{t+2}, \dots, Y_{s-1}\}$ removed (something like a "conditional autocorrelation").

In [ ]:
def plot_acf_pacf(x, nlags=25):
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))

    plot_acf(x, lags=nlags, ax=ax[0])
    plot_pacf(x, lags=nlags, ax=ax[1])

    return fig
    

**White noise**

White noise shouldn't have any correlation in either the ACF or the PACF

In [ ]:
plot_acf_pacf(wn, nlags=25);

**Autoregressive process**

The $\tau$ lag correlation between two observations would be given by $\rho^{\tau}$

In [ ]:
# Notice now that all correlation is assigned
# to the tau=1 lag -- This is because if we
# knew Y_{t-1} then we'd have all the relevant
# correlations for thinking about Y_t
plot_acf_pacf(ar, nlags=25);

**Random walk**

The correlation of the random walk would be $\sqrt{\frac{1}{s}}$

In [ ]:
plot_acf_pacf(rw, nlags=25);

**Scatter plots**

Scatter plots are great for thinking about time series data... We highly recommend them.

In [ ]:
def plot_ts_scatter(obs):
    # t values
    x = obs[0:-1]

    # t+1 values
    y = obs[1:]

    fig, ax = plt.subplots(figsize=(4, 4))
    ax.scatter(x, y)

    return fig

In [ ]:
plot_ts_scatter(wn);

In [ ]:
plot_ts_scatter(ar);

In [ ]:
plot_ts_scatter(rw);

## Moving average processes

The first time series model that we'll discuss today is the moving average model (MA):

$$Y_t = \sum_{j=0}^\infty \phi_j \varepsilon_{t-j}$$

where $\varepsilon_t \sim N(0, \sigma)$ and $\{\phi_j\}$ is square-summable ($\sum_{j=0}^\infty \phi_j^2 < \infty$)

We will often refer to the moving average by the number of non-zero coefficients -- i.e., an MA(q) would be given by:

$$Y_t = \sum_{j=1}^q \phi_j \varepsilon_{t-j} + \sigma \varepsilon_t$$

### Why do we care about this particular process?

This particular process is very powerful because of the [Wold representation theorem](https://en.wikipedia.org/wiki/Wold%27s_theorem) which states that,

> Every zero-mean stationary process, $\{Y_t\}$, can be written as $Y_t = \eta_t + \sum_{j=0}^\infty \phi_j \varepsilon_{t-j}$ where $\eta_t$ is entirely deterministic



**White noise?**

White noise can be written using an MA process...

This one is "easy" to guess the right coefficients:

$$\phi_j = \begin{cases} 1 \; j=0 \\ 0 \; j \neq 0 \end{cases}$$

**Autoregressive processes?**

We'll think through an AR(1) but similar algebra would allow us to write any autoregressive process as a MA.

\begin{align*}
  Y_t &= \rho Y_{t-1} + \varepsilon_t \\
  &= \rho (\rho Y_{t-2} + \varepsilon_{t-1}) + \varepsilon_t \\
  &= \dots \\
  &= \sum_{j=0}^\infty \rho^j \varepsilon_{t-j}
\end{align*}

thus $\phi_j = \rho^j$

**Any process?**

Yep. We will skip the proof today, but any process that is weakly stationary (constant mean and autocovariance that only depends on the lag) can be written using an MA process

### Simulating a MA

In [ ]:
@numba.jit(nopython=True)
def simulate_ma(T, phis, sigma):
    # Compute the order of MA that we're using
    q = len(phis)

    # Allocate space
    epsilons = sigma*np.random.randn(T+q)
    out = np.zeros(T)
    for t in range(T):
        value = 0.0
        for j in range(q):
            value += phis[j]*epsilons[q+t-j]
        out[t] = value

    return out

In [ ]:
phis = np.array([1.0, 0.75, 0.75, 0.5, 0.5, 0.3, 0.3])
ma = simulate_ma(1000, phis, 0.1)

In [ ]:
plt.plot(ma)

In [ ]:
plot_ts_scatter(ma);

In [ ]:
# Which plot is more helpful for understanding the MA coeffs?
plot_acf_pacf(ma);

### Estimating a MA process

The MA model isn't fit using regression -- The $\varepsilon_t$ values are unobserved and so it is done via a non-linear fitting procedure (maximum likelihood for example).

**Choosing number of lags**

One way to choose a good first guess at the right number of lags is to look at the autocorrelation function.

Pick the $q$ such that the autocorrelations at higher lags are zero, i.e. $q$ such that $\rho_s \approx 0$  for $s > q$

In [ ]:
# Pick q = 6
plot_acf_pacf(ma);

In [ ]:
# Fit the model using statsmodels
ma_model = ARIMA(ma, order=(0, 0, 6))
res_ma = ma_model.fit()

print(res_ma.summary())

### Forecasting a MA process

After fitting a MA model, we have access to a variety of methods that we can apply to the fitted model.

One of these methods is `get_forecast`

In [ ]:
nforecast = 50

forecast_ma = res_ma.get_forecast(nforecast)
 
mf = forecast_ma.predicted_mean
ci = forecast_ma.conf_int()

In [ ]:
fig, ax = plt.subplots()

tvalues = np.arange(100 + nforecast)

ax.plot(tvalues[:100], ma[-100:], color="k")
ax.plot(tvalues[100:], mf, color="b", linestyle="--")

ax.fill_between(tvalues[100:], ci[:, 0], ci[:, 1], alpha=0.5)

## Autoregressive processes

While MA processes have great theoretical properties, sometimes it is convenient to work with AR processes...

The generic AR(p) process is written by:

$$Y_{t+1} = \left( \sum_{j=0}^p \rho_j Y_{t-j} \right) + \sigma \varepsilon_{t+1}$$

Reasons why it can be convenient to work with an AR process rather than an MA:

* If the data generating process is an AR(1) then an MA requires an infinite number of parameters while the AR only requires one...
* AR models are linear in observed variables which makes regression easier

### Simulating an AR process



In [ ]:
@numba.jit(nopython=True)
def simulate_ar(T, rhos, sigma):
    # Compute the order of AR that we're using
    p = len(rhos)

    # Allocate space
    out = np.zeros(T + 2*p)
    for t in range(p, T + 2*p):
        value = 0.0
        for j in range(p):
            value += rhos[j]*out[t-j-1]
        out[t] = value + sigma*np.random.randn()

    return out[2*p:]

In [ ]:
ar = simulate_ar(1000, np.array([0.7, 0.3, -0.2]), 0.1)

In [ ]:
plt.plot(ar)

In [ ]:
plot_ts_scatter(ar);

In [ ]:
plot_acf_pacf(ar);

### Estimating an AR process

Autoregressive models fit the requirements of a regression, so we can regress $\{Y_t\}$ on its lags

**How many lags?**

One way to choose a good first guess at the right number of lags is to look at the partial autocorrelation function.

Pick the $p$ such that the partial autocorrelations at higher lags are zero, i.e. $p$ such that $\alpha_s \approx 0$  for $s > p$

In [ ]:
# Pick p = 3
plot_acf_pacf(ar);

In [ ]:
# Fit the model using statsmodels
ar_model = ARIMA(ar, order=(3, 0, 0))
res_ar = ar_model.fit()

print(res_ar.summary())

### Forecasting an AR process

In [ ]:
nforecast = 50

forecast_ar = res_ar.get_forecast(nforecast)

mf = forecast_ar.predicted_mean
ci = forecast_ar.conf_int()

In [ ]:
fig, ax = plt.subplots()

tvalues = np.arange(100 + nforecast)

ax.plot(tvalues[:100], ar[-100:], color="k")
ax.plot(tvalues[100:], mf, color="b", linestyle="--")

ax.fill_between(tvalues[100:], ci[:, 0], ci[:, 1], alpha=0.5)

## ARMA / ARIMA

We now present two additions to the models we've seen previously:

We define an ARMA(p, q) model as the mixture of an AR(p) and MA(q) model

$$Y_{t+1} = \left( \sum_{j=0}^p \rho_j Y_{t-j} \right) + \left( \sum_{j=0}^q \phi_j \sigma \varepsilon_{t-j} \right) + \sigma \varepsilon_{t+1}$$

In order to discuss the ARIMA model, we need to refresh our memory on the lag operator ($L$).

\begin{align*}
  L x_t &= x_{t-1} \\
  L^2 x_t &= x_{t-2} \\
  \dots
\end{align*}

then we define an ARIMA(p, d, q) model as the mixture of an AR(p) and MA(q) model with differencing (to help make the process stationary)

$$(1 - \sum_{j=0}^p \rho_j L^j) (1 - L)^d Y_{t} = \delta + (1 + \sum_{j=0}^q \phi_j L^j) \sigma \varepsilon_t$$

It's best to see what this results in when we set $d = 1$.

$$Y_{t+1} - Y_t = \sum_{j=0}^p \rho_j (Y_{t-j} - Y_{t-j-1}) + \sum_{j=0}^q \phi_j \sigma \varepsilon_{t-j} + \sigma \varepsilon_{t+1}$$

### Estimating and forecasting ARIMA process

Picking a $p$, $d$, and $q$ is not always obvious...

There are [packages](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html) that help with picking the parameters but they basically boil down to "brute force" selection

In [ ]:
from pandas_datareader import DataReader

# Lets load US GDP data
gdp = DataReader("GDP", "fred", 1975, 2021)

In [ ]:
gdp.plot()

In [ ]:
def fit_ARIMA(train, test, p, d, q):
    ntrain = train.shape[0]
    ntest = test.shape[0]

    # Fit model
    m = ARIMA(train.to_numpy(), order=(p, d, q))
    res = m.fit()
    print(res.summary())

    forecast = res.get_forecast(ntest)
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()

    # Make plots
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))

    ax[0].plot(train.index, train.to_numpy(), color="k", linewidth=1.5)
    ax[0].plot(test.index, test.to_numpy(), color="k", linewidth=1.5, linestyle="--")
    ax[0].plot(test.index, forecast_mean, color="DarkGreen", linewidth=1, linestyle="--")
    ax[0].fill_between(test.index, forecast_ci[:, 0], forecast_ci[:, 1], color="DarkGreen", alpha=0.45)
    ax[0].set_title("Forecast")

    ax[1].plot(test.index, test.to_numpy()[:, 0] - forecast_mean)
    ax[1].set_title("Difference")

    return fig

In [ ]:
train = gdp.loc[:"2008"]
test = gdp.loc["2009":"2010"]


fit_ARIMA(train, test, 1, 1, 1);